# Creating a dataset for protein binders using a 3D representation

This tutorial shows how to create a dataset for protein binders. We want to represent 3D features of different conformers of the molecule, but there are many different conformers for a given molecules. So, we extract a set of conformers and consider their Boltzmann weights.

### Set up Django

In [1]:
import os
import django

import sys

# Make sure htvs/djangochem is in your path!
sys.path.insert(0, "/home/saxelrod/htvs")
sys.path.insert(0, "/home/saxelrod/htvs/djangochem")

os.environ["DJANGO_SETTINGS_MODULE"]="djangochem.settings.orgel"


django.setup()

# Shell Plus Model Imports
from features.models import AtomDescriptor, BondDescriptor, ConnectivityMatrix, DistanceMatrix, Fingerprint, ProximityMatrix, SpeciesDescriptor, TrainingSet, Transformation
from guardian.models import GroupObjectPermission, UserObjectPermission
from django.contrib.contenttypes.models import ContentType
from neuralnet.models import ActiveLearningLoop, NetArchitecture, NetCommunity, NetFamily, NeuralNetwork, NnPotential, NnPotentialStats
from jobs.models import Job, JobConfig, WorkBatch
from django.contrib.admin.models import LogEntry
from django.contrib.auth.models import Group, Permission, User
from django.contrib.sessions.models import Session
from pgmols.models import (AtomBasis, BasisSet, Batch, Calc, Cluster,
                           Geom, Hessian, Jacobian, MDFrame, Mechanism, Method, Mol, MolGroupObjectPermission,
                           MolSet, MolUserObjectPermission, PathImage, ProductLink, ReactantLink, Reaction,
                           ReactionPath, ReactionType, SinglePoint, Species, Stoichiometry, Trajectory)
# Shell Plus Django Imports
from django.core.cache import cache
from django.db import transaction
from django.utils import timezone
from django.contrib.auth import get_user_model
from django.urls import reverse
from django.conf import settings
from django.db.models import Avg, Case, Count, F, Max, Min, Prefetch, Q, Sum, When, Exists, OuterRef, Subquery

RDKit WARNING: [19:27:06] Enabling RDKit 2019.09.1 jupyter extensions


## MMFF94

Get a dataset of potential covid binders where the conformers are generated with mmff94

In [2]:
from neuralnet.utils.nff import create_bind_dataset

group_name = 'covid'
method_name = 'molecular_mechanics_mmff94'
method_descrip = 'MMFF conformer.'
molsets = ['run']
nbrlist_cutoff = 5.0
batch_size = 10
num_workers = 2
# maximum conformers per species
geoms_per_spec = 10

dataset, loader = create_bind_dataset(group_name=group_name,
                    method_name=method_name,
                    method_descrip=method_descrip,
                    geoms_per_spec=geoms_per_spec,
                    nbrlist_cutoff=nbrlist_cutoff,
                    batch_size=batch_size,
                    num_workers=num_workers,
                    molsets=molsets)

dataset.save('covid_mmff94.pth.tar')



## Crest
Same, but with crest (much better!)

In [3]:
from neuralnet.utils.nff import create_bind_dataset



method_name = 'gfn2-xtb'
method_descrip = 'Crest GFN2-xTB'

dataset, loader = create_bind_dataset(group_name=group_name,
                    method_name=method_name,
                    method_descrip=method_descrip,
                    geoms_per_spec=geoms_per_spec,
                    nbrlist_cutoff=nbrlist_cutoff,
                    batch_size=batch_size,
                    num_workers=num_workers,
                    molsets=molsets)

dataset.save('covid_crest.pth.tar')


## The dataset

Let's take a look at the dataset itself:

In [4]:
print(dataset[0])

{'weights': tensor([0.1814, 0.1683, 0.1215, 0.0636, 0.0582, 0.1142, 0.0527, 0.0517, 0.1492,
        0.0393]), 'bind': tensor(0), 'mol_size': tensor(43), 'num_atoms': tensor(430), 'spec_id': tensor(6631940), 'nxyz': tensor([[ 6.0000,  4.3809,  1.0873,  0.1777],
        [ 6.0000,  3.8217,  0.4284, -0.9035],
        [ 6.0000,  2.4735,  0.5657, -1.1796],
        ...,
        [ 1.0000, -1.0070, -2.4155,  1.4414],
        [ 1.0000, -3.3377, -2.3545,  2.2281],
        [ 1.0000, -5.0852, -1.3165,  0.8359]]), 'smiles': 'c1ccc(CC(c2ccccc2)N2CCCCC2)cc1', 'nbr_list': tensor([[  0,   1],
        [  0,   2],
        [  0,   3],
        ...,
        [429, 394],
        [429, 427],
        [429, 428]])}


Each item in the dataset corresponds to one species. It has the number of atoms (`num_atoms`), whether it's a binder (`bind`), its smiles, the database IDs of its conformer geoms, the database ID of the species. It also has `mol_size`, the actual number of atoms in one molecule, which will allow us to separate the big nxyz (a stacked tensor consisting of all conformer nxyz's) into its conformers when needed. `weights` is a list of Boltzmann weights fo reach conformer. `nbr_list` tells you the neighbors of each atom, which takes into account the fact that ever 43 atoms you're actually in a different molecule.

Let's next look at batching:

In [5]:
print(next(iter(loader)))

{'weights': tensor([1.8140e-01, 1.6827e-01, 1.2152e-01, 6.3598e-02, 5.8217e-02, 1.1422e-01,
        5.2662e-02, 5.1665e-02, 1.4919e-01, 3.9260e-02, 8.2947e-01, 1.6617e-01,
        1.4203e-03, 1.4003e-03, 6.4012e-04, 5.2009e-04, 5.0009e-05, 2.0004e-05,
        2.4004e-04, 7.0013e-05, 3.5503e-01, 2.4643e-01, 1.2268e-01, 8.3726e-02,
        5.5241e-02, 5.5166e-02, 2.6927e-02, 2.2071e-02, 1.7791e-02, 1.4941e-02,
        4.1169e-01, 5.3856e-01, 1.8871e-02, 1.9101e-03, 2.7221e-02, 2.1001e-04,
        2.4001e-04, 2.3001e-04, 1.0200e-03, 5.0002e-05, 1.7496e-01, 9.2255e-02,
        9.3527e-02, 4.0222e-02, 2.5229e-01, 6.6510e-02, 1.2409e-01, 2.7637e-02,
        7.9095e-02, 4.9412e-02, 9.3817e-01, 1.2143e-02, 4.3072e-02, 3.9211e-03,
        2.1006e-04, 1.2303e-03, 7.8022e-04, 2.2006e-04, 1.9005e-04, 6.0017e-05,
        1.2285e-01, 3.9031e-02, 3.2004e-01, 1.7916e-01, 6.9935e-02, 3.5137e-02,
        3.4967e-02, 6.6633e-02, 9.8044e-02, 3.4205e-02, 3.3080e-01, 1.6296e-01,
        8.2121e-02, 6.5205e-

This looks the exact same as a regular batch, if we assumed that each smiles really had one giant xyz.